In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
# Function with for loop to download all stats of the team internal - Dict as I called teams differently to the website
teams_internal = {'LAL':'LAL','CHI':'CHB','GSW':'GSW','MIA':'MIH','BOS':'BOC', 'BRK':'BRK', 'PHI':'PHI', 'MIL':'MIL', 'NYK':'NYK', 'ATL':'ATL', 'CHO':'CHO', 'IND':'IND', 'WAS':'WAS' , 'TOR':'TOR', 'CLE':'CLE', 'ORL':'ORL', 'DET':'DET', 'UTA':'UTA', 'PHO':'PHO', 'LAC':'LAC', 'DEN':'DEN', 'DAL':'DAL', 'POR':'POR', 'MEM':'MEM', 'SAS':'SAS', 'NOP':'NOP', 'SAC':'SAC', 'OKC':'OKC', 'MIN':'MIN', 'HOU':'HOU'}
# only for relevant teams
#teams_internal = {'LAL':'LAL','CHI':'CHB','GSW':'GSW','MIA':'MIH','BOS':'BOC'}

def scrape_team_per_game(year):
    # this is the HTML from the given URL
    for team in teams_internal:
         # create the url for the different team i want to scrape their historical stats of
        url = 'https://www.basketball-reference.com/teams/'+ str(team) + '/' + str(year) + '/gamelog/'


        # this is the HTML from the given URL
        html = urlopen(url)
        soup = BeautifulSoup(html)

        # use findALL() to get the column headers
        soup.findAll('tr', limit=2)

        # use getText()to extract the text we need into a list
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
        headers[3] = 'Home / Away'
        headers[-17] = 'to_be_filled'
        headers[7] = 'P_Opp'
        headers[6] = 'P_Team'

        # exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
        headers_notRK = headers[1:]
        headers_notRK

        # avoid the first header row
        rows = soup.findAll('tr')[2:]

        stats_scraped = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

        # Convert to Dataframe
        stats_df = pd.DataFrame(stats_scraped, columns = headers_notRK)

        # drop row as there is no information in it
        stats_df = stats_df.drop(columns = ['to_be_filled'])
        stats_df_team = stats_df.iloc[:,: 23]
        stats_df_team.head(1)

        # Replacing @ to home and nothing to away
        stats_df_team.loc[stats_df_team['Home / Away'] == '@', 'Home / Away'] = 'Away'
        stats_df_team.loc[stats_df_team['Home / Away'] == '', 'Home / Away'] = 'Home'

        # Delete empty rows
        stats_df_team.dropna(axis=0,how='all', thresh=None, subset=None, inplace=True)

        # Add home time and season label
        stats_df_team['Home Team'] = str(teams_internal[team])
        stats_df_team['Season'] = year

        # make number numeric
        stats_df_team[['P_Team', 'P_Opp', 'FG',
               'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
               'AST', 'STL', 'BLK', 'TOV', 'PF']] = stats_df_team[['P_Team', 'P_Opp', 'FG',
               'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
               'AST', 'STL', 'BLK', 'TOV', 'PF']].apply(pd.to_numeric, errors='coerce')
        #create file with stats
        stats_df_team.to_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/' + str(teams_internal[team]) + '/01_stats_original/' + str(teams_internal[team]) + '_' + str(year) + '_historical_stats.xlsx', index = False)

In [3]:
# helper function, built into get averages_stats
# five game running average for nummerical features
def five_game_avg(stats):
    # clean df_stats
    # Drop percentages as they can not be added (newly calculated later)
    stats.drop(columns = ['FG%', '3P%', 'FT%'], inplace = True)

    # Moving Home Team to the front
    stats = stats[['G', 'Home Team', 'Date', 'Home / Away', 'Opp', 'W/L', 'P_Team', 'P_Opp', 'FG',
           'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK',
           'TOV', 'PF']]

    # splitting numeric columns
    # predicting only after first x game good, as the average will be less biased on 1 occurance
    stats_num = stats[['P_Team', 'P_Opp', 'FG',
           'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK',
           'TOV', 'PF']]


    # Append first row as there cannot be an average
    stats_num_avg = stats_num.iloc[[0]]

    # Append average of the first rows as they cannot display a 5 games average
    for i in range(2,5):   ### 2, as iloc is until 2 (excluding 2), also therefore + 1
        stats_num_help = stats_num.iloc[:i]
        stats_num_help2 = pd.DataFrame(stats_num_help.mean(axis = 0)).T
        stats_num_avg = stats_num_avg.append(stats_num_help2)

    # Append 5 game running average
    for i in range(5,len(stats_num) + 1):   ### 2, as iloc is until 2 (excluding 2), also therefore + 1
        stats_num_help = stats_num.iloc[(i-5):i]
        stats_num_help2 = pd.DataFrame(stats_num_help.mean(axis = 0)).T
        stats_num_avg = stats_num_avg.append(stats_num_help2)

    # Create percentages once again
    stats_num_avg['FG%'] = stats_num_avg['FG'] / stats_num_avg['FGA']
    stats_num_avg['3P%'] = stats_num_avg['3P'] / stats_num_avg['3PA']
    stats_num_avg['FT%'] = stats_num_avg['FT'] / stats_num_avg['FTA']

    stats_num_avg.reset_index(inplace = True)
    stats_num_avg = stats_num_avg.drop(columns = ['index'])

    # adding suffix to features for 5 Game average
    stats_num_avg.columns = ['P_Team_5GA', 'P_Rec_5GA', 'FG_5GA', 'FGA_5GA', '3P_5GA', '3PA_5GA', 'FT_5GA', 'FTA_5GA','ORB_5GA', 'TRB_5GA', 'AST_5GA', 'STL_5GA', 'BLK_5GA', 'TOV_5GA', 'PF_5GA', 'FG%_5GA', '3P%_5GA', 'FT%_5GA']

    return stats_num_avg

In [4]:
# creating the training dataset where we see the cumulative average scores up until every game
# containing five game average function
def get_averages_stats(year):
        for team in teams_internal:

            # retrieve stats per team
            stats = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ str(teams_internal[team]) + '/01_stats_original/'+ str(teams_internal[team]) + '_'+ str(year) + '_historical_stats.xlsx')

            # Drop percentages as they can not be added (newly calculated later)
            stats.drop(columns = ['FG%', '3P%', 'FT%'], inplace = True)

            # Moving Home Team to the front
            stats = stats[['G', 'Home Team', 'Date', 'Home / Away', 'Opp', 'W/L', 'P_Team', 'P_Opp', 'FG',
                   'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK',
                   'TOV', 'PF']]

            # splitting numeric columns
            # predicting only after first x game good, as the average will be less biased on 1 occurance
            stats_num = stats[['P_Team', 'P_Opp', 'FG',
                   'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK',
                   'TOV', 'PF']]

            # Loop to add all averages
            stats_num_avg = pd.DataFrame(columns = ['P_Team', 'P_Opp', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB','AST', 'STL', 'BLK', 'TOV', 'PF'])

            # Append first row as there cannot be an average
            stats_num_avg = stats_num.iloc[[0]]

            # Append average of the first i
            for i in range(2,len(stats_num) + 1):
                stats_num_help = stats_num.iloc[:i]
                stats_num_help2 = pd.DataFrame(stats_num_help.mean(axis = 0)).T
                stats_num_avg = stats_num_avg.append(stats_num_help2)

            # Create percentages once again
            stats_num_avg['FG%'] = stats_num_avg['FG'] / stats_num_avg['FGA']
            stats_num_avg['3P%'] = stats_num_avg['3P'] / stats_num_avg['3PA']
            stats_num_avg['FT%'] = stats_num_avg['FT'] / stats_num_avg['FTA']

            stats_num_avg.reset_index(inplace = True)


            ##########  for non nummeric data (win/loss streak)

            # retrieve stats per team
            stats = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ str(teams_internal[team]) + '/01_stats_original/'+ str(teams_internal[team]) + '_'+ str(year) + '_historical_stats.xlsx')

            # splitting non-numeric columns
            stats_cat = stats[['G', 'Home Team', 'Date', 'Home / Away', 'Opp', 'W/L']]

            # create win and loss streaks
            wins = stats_cat['W/L']
            losses = stats_cat['W/L']

            wins = pd.DataFrame(wins)
            wins = wins.rename(columns = {'W/L': 'Wins'})

            losses = pd.DataFrame(losses)
            losses = losses.rename(columns = {'W/L': 'Losses'})

            result = pd.concat([wins,losses], axis=1)

            result.Wins.replace('L', 0, inplace = True)
            result.Wins.replace('W', 1, inplace = True)
            result.Losses.replace('L', 1, inplace = True)
            result.Losses.replace('W', 0, inplace = True)
            result.Losses.replace('W', 0, inplace = True)

            result['Losses_streak'] = result.Losses.astype(bool)
            result['Wins_streak'] = result.Wins.astype(bool)

            result['Wins_streak'] = (result['Wins_streak'].groupby([result['Wins_streak'], (~result['Wins_streak']).cumsum().where(result['Wins_streak'])]).cumcount().add(1).mul(result['Wins_streak']))
            result['Losses_streak'] = (result['Losses_streak'].groupby([result['Losses_streak'], (~result['Losses_streak']).cumsum().where(result['Losses_streak'])]).cumcount().add(1).mul(result['Losses_streak']))
            result = result.drop(columns = ['Wins', 'Losses'])

            # add win / loss streak back to non_num table
            non_num = stats[['G', 'Home Team', 'Date', 'Home / Away', 'Opp', 'W/L']]
            pre_final = pd.concat([non_num, result], axis = 1)

            # append to num num table
            final = pd.concat([pre_final, stats_num_avg], axis = 1)

            #rename P_opp to P_received
            final = final.rename(columns = {'P_Opp' : 'P_Received'})

            #drop index column that was in there because of concatination
            final = final.drop(columns = ['index'])

            # append five year running get_averages_stats
            stats_for_avg = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ str(teams_internal[team]) + '/01_stats_original/'+ str(teams_internal[team]) + '_'+ str(year) + '_historical_stats.xlsx')
            df_avg = five_game_avg(stats_for_avg)
            final2 = pd.concat([final, df_avg], axis = 1)

            ##### create win percentages
            df = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ str(teams_internal[team]) + '/01_stats_original/'+ str(teams_internal[team]) + '_'+ str(year) + '_historical_stats.xlsx')

            # for whole season
            win_per = []

            for i in range(1, len(df)+1):

                # calculate winning percentages
                if len(df.iloc[:i].groupby(['W/L']).count()['G']) > 1:
                    new_win_per = df.iloc[:i].groupby(['W/L']).count()['G'].iloc[1] / len(df.iloc[:i])
                else: new_win_per = 0

                # Append to list
                win_per.append(new_win_per)


            # for past 5 games
            win_per_5g = []

            for i in range(1,5):

                # calculate winning percentages
                if len(df.iloc[:i].groupby(['W/L']).count()['G']) > 1:
                    new_win_per = df.iloc[:i].groupby(['W/L']).count()['G'].iloc[1] / len(df.iloc[:i])
                elif df.iloc[:i].groupby(['W/L']).count()['G'].index == 'W':
                    new_win_per = 1
                else: new_win_per = 0

                # Append to list
                win_per_5g.append(new_win_per)


            for i in range(5, len(df)+1):

                # calculate winning percentages
                if len(df.iloc[(i-5):i].groupby(['W/L']).count()['G']) > 1:
                    new_win_per = df.iloc[(i-5):i].groupby(['W/L']).count()['G'].iloc[1] / 5
                elif df.iloc[(i-5):i].groupby(['W/L']).count()['G'].index == 'W':
                    new_win_per = 1
                else: new_win_per = 0

                # Append to list
                win_per_5g.append(new_win_per)


            df_win_per = pd.DataFrame({'win_per': win_per, 'win_per_5G': win_per_5g})


            final2 = final2.join(df_win_per, how = 'left', on = df.index)


            ######### Deleting the first game as there are no historical stats yet - for the second game stats of the first game

            #final2 = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ str(teams_internal[team]) + '/02_stats_average/'+ str(teams_internal[team]) + '_'+ str(year) + '_stats_num_avg.xlsx')

            l_hand = final2.iloc[1:,:6].reset_index()
            r_hand = final2.iloc[:71,6:].reset_index()

            final3 = pd.concat([l_hand, r_hand], axis = 1).drop(columns = ['index'])

            #final3 = final3.drop(columns = ['days_rest'])

            ######### Days rest feature

            df.Date = pd.to_datetime(df.Date, format = '%Y-%m-%d')

            days_rest = []

            # second+ rows
            for i in range(1, len(df)):
                days_rest.append((df.Date.iloc[i]-df.Date.iloc[i-1]).days)

            df_days_rest = pd.DataFrame({'days_rest': days_rest})


            final3 = final3.join(df_days_rest, how = 'left')

            # write excel
            final3.to_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ str(teams_internal[team]) + '/02_stats_average/'+ str(teams_internal[team]) + '_'+ str(year) + '_stats_num_avg.xlsx', index = False)

Notes:
- Url: https://www.basketball-reference.com/teams/CHI/2021/gamelog/
- Only retrieves the stats from the home team

---


In [5]:
# scraping and transofrming cumulative averages 2000 - present
for i in range(2021, 2022):
    # Scraping the whole year
    scrape_team_per_game(i)
    # creating cumulative averages
    get_averages_stats(i)

    print('Done')

IndentationError: unexpected indent (<ipython-input-5-1d548fe27bbd>, line 4)